In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.utils import save_image
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Updated Data Loading and Preprocessing
class SignatureDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.images = []
        self.labels = []
        
        for class_name in self.classes:
            class_path = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_path):
                self.images.append(os.path.join(class_path, img_name))
                self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('L')  # Convert to grayscale
        if self.transform:
            image = self.transform(image)
        return image, label

In [4]:
# Data Augmentation
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
])

# Load and split the dataset
def load_and_split_data(root_dir, train_ratio=0.8):
    full_dataset = SignatureDataset(root_dir=root_dir, transform=transform)
    train_size = int(train_ratio * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])
    return train_dataset, test_dataset

In [5]:
# VAE Implementation (unchanged)
class VAE(nn.Module):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.ReLU(),
            nn.Flatten()
        )
        
        self.fc_mu = nn.Linear(256 * 4 * 4, latent_dim)
        self.fc_logvar = nn.Linear(256 * 4 * 4, latent_dim)
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256 * 4 * 4),
            nn.ReLU(),
            nn.Unflatten(1, (256, 4, 4)),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 4, 2, 1),
            nn.Sigmoid()
        )
        
    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [6]:
# GAN Implementation (unchanged)
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256 * 4 * 4),
            nn.ReLU(),
            nn.Unflatten(1, (256, 4, 4)),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 4, 2, 1),
            nn.Tanh()
        )
    
    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

In [7]:
# Training functions (slightly modified)
def train_vae(vae, optimizer, train_loader, num_epochs=100):
    vae.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch, _ in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = vae(batch)
            loss = vae_loss(recon_batch, batch, mu, logvar)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}')

def train_gan(generator, discriminator, g_optimizer, d_optimizer, train_loader, num_epochs=100):
    criterion = nn.BCELoss()
    for epoch in range(num_epochs):
        for batch, _ in train_loader:
            batch = batch.to(device)
            batch_size = batch.size(0)
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)

            # Train Discriminator
            d_optimizer.zero_grad()
            outputs = discriminator(batch)
            d_loss_real = criterion(outputs, real_labels)
            d_loss_real.backward()

            noise = torch.randn(batch_size, generator.latent_dim).to(device)
            fake_images = generator(noise)
            outputs = discriminator(fake_images.detach())
            d_loss_fake = criterion(outputs, fake_labels)
            d_loss_fake.backward()
            d_loss = d_loss_real + d_loss_fake
            d_optimizer.step()

            # Train Generator
            g_optimizer.zero_grad()
            outputs = discriminator(fake_images)
            g_loss = criterion(outputs, real_labels)
            g_loss.backward()
            g_optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}')

def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [8]:
# Generating New Signatures (updated for group-based generation)
def generate_vae_signatures(vae, num_samples=10, group_idx=None):
    vae.eval()
    with torch.no_grad():
        if group_idx is not None:
            # Implement group-specific latent space sampling
            z = torch.randn(num_samples, vae.latent_dim).to(device) + group_idx
        else:
            z = torch.randn(num_samples, vae.latent_dim).to(device)
        samples = vae.decode(z)
    return samples

def generate_gan_signatures(generator, num_samples=10, group_idx=None):
    generator.eval()
    with torch.no_grad():
        noise = torch.randn(num_samples, generator.latent_dim).to(device)
        if group_idx is not None:
            z = noise + group_idx
        else:
            z = noise
        samples = generator(noise)
    return samples

# Metrics and Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch, _ in test_loader:
            batch = batch.to(device)
            recon_batch, mu, logvar = model(batch)
            loss = vae_loss(recon_batch, batch, mu, logvar)
            total_loss += loss.item()
    return total_loss / len(test_loader)

In [18]:
latent_dim = 500
root_dir = 'Grouped_Output'  

In [10]:
# Load and split the data
train_dataset, test_dataset = load_and_split_data(root_dir)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)   

In [11]:
# VAE training
vae = VAE(latent_dim).to(device)
vae_optimizer = optim.Adam(vae.parameters())
train_vae(vae, vae_optimizer, train_loader,num_epochs=1500)

Epoch 1, Loss: 161871.2263
Epoch 2, Loss: 158686.8174
Epoch 3, Loss: 157970.8489
Epoch 4, Loss: 157717.4922
Epoch 5, Loss: 157427.9253
Epoch 6, Loss: 157091.2727
Epoch 7, Loss: 156880.4265
Epoch 8, Loss: 156467.2346
Epoch 9, Loss: 156336.6587
Epoch 10, Loss: 156274.1013
Epoch 11, Loss: 156008.1587
Epoch 12, Loss: 155946.6541
Epoch 13, Loss: 155879.2517
Epoch 14, Loss: 154434.5994
Epoch 15, Loss: 152120.7046
Epoch 16, Loss: 149678.5898
Epoch 17, Loss: 147898.2937
Epoch 18, Loss: 145935.8523
Epoch 19, Loss: 144906.4243
Epoch 20, Loss: 144947.3774
Epoch 21, Loss: 144857.1812
Epoch 22, Loss: 144265.2004
Epoch 23, Loss: 143829.6934
Epoch 24, Loss: 144609.1226
Epoch 25, Loss: 143458.5984
Epoch 26, Loss: 143706.9707
Epoch 27, Loss: 143463.1028
Epoch 28, Loss: 143047.3708
Epoch 29, Loss: 142835.1455
Epoch 30, Loss: 142861.5154
Epoch 31, Loss: 143585.5120
Epoch 32, Loss: 143580.1409
Epoch 33, Loss: 142402.0161
Epoch 34, Loss: 142572.9077
Epoch 35, Loss: 143058.9792
Epoch 36, Loss: 142968.9067
E

In [12]:
# GAN training
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
train_gan(generator, discriminator, g_optimizer, d_optimizer, train_loader,num_epochs=2500)

Epoch [1/100], d_loss: 0.1232, g_loss: 3.7374
Epoch [2/100], d_loss: 0.0535, g_loss: 4.8415
Epoch [3/100], d_loss: 0.0260, g_loss: 5.3070
Epoch [4/100], d_loss: 0.0150, g_loss: 5.9209
Epoch [5/100], d_loss: 0.0111, g_loss: 6.3410
Epoch [6/100], d_loss: 0.0084, g_loss: 6.3041
Epoch [7/100], d_loss: 0.0060, g_loss: 6.5305
Epoch [8/100], d_loss: 0.0066, g_loss: 6.6569
Epoch [9/100], d_loss: 0.0054, g_loss: 6.8628
Epoch [10/100], d_loss: 0.0089, g_loss: 6.8088
Epoch [11/100], d_loss: 0.0054, g_loss: 6.9921
Epoch [12/100], d_loss: 0.0041, g_loss: 6.8431
Epoch [13/100], d_loss: 0.0038, g_loss: 7.0618
Epoch [14/100], d_loss: 0.0041, g_loss: 7.1070
Epoch [15/100], d_loss: 0.0048, g_loss: 7.1594
Epoch [16/100], d_loss: 0.0034, g_loss: 7.3429
Epoch [17/100], d_loss: 0.0023, g_loss: 7.4777
Epoch [18/100], d_loss: 0.0060, g_loss: 6.7932
Epoch [19/100], d_loss: 0.0016, g_loss: 7.8731
Epoch [20/100], d_loss: 0.0022, g_loss: 7.9461
Epoch [21/100], d_loss: 0.0062, g_loss: 15.0403
Epoch [22/100], d_los

In [13]:
# Generate new signatures
vae_samples = generate_vae_signatures(vae)
gan_samples = generate_gan_signatures(generator)

In [14]:
# Save generated samples
save_image(vae_samples, 'Output/vae_generated_signatures.png')
save_image(gan_samples, 'Output/gan_generated_signatures.png')

In [15]:
# Evaluate models
vae_loss = evaluate_model(vae, test_loader)
print(f'VAE Test Loss: {vae_loss:.4f}')

VAE Test Loss: 141480.6172


In [17]:
# Generate group-specific signatures (placeholder)
num_groups = len(train_dataset.dataset.classes)
for group_idx in range(num_groups):
    group_name = train_dataset.dataset.classes[group_idx]
    vae_group_samples = generate_vae_signatures(vae, num_samples=1, group_idx=group_idx)
    gan_group_samples = generate_gan_signatures(generator, num_samples=1, group_idx=group_idx)
    os.makedirs("Output/"+group_name, exist_ok=True)
    save_image(vae_group_samples, f'Output/{group_name}/vae_generated_signatures_group_{group_name}.png')
    save_image(gan_group_samples, f'Output/{group_name}/gan_generated_signatures_group_{group_name}.png')

print("Training, evaluation, and group-specific generation complete!")

Training, evaluation, and group-specific generation complete!
